In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.datasets import mnist
from keras.utils import to_categorical

from keras import models
from keras import layers

from keras import losses, optimizers, metrics

In [5]:
from keras.preprocessing.image import ImageDataGenerator
train_dir = 'data\\train'
test_dir ='data\\test'


train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (224, 224),
    batch_size=32
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (224, 224),
    batch_size=8
)

Found 4651 images belonging to 7 classes.
Found 247 images belonging to 7 classes.


# Try using pretrined model

In [2]:
from keras.applications import VGG19

conv_base = VGG19(weights='imagenet',
                 include_top=False,
                 input_shape=(224, 224, 3))

conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [6]:
model = models.Sequential()

model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512))
model.add(layers.Dense(7, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               12845568  
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 3591      
Total params: 32,873,543
Trainable params: 32,873,543
Non-trainable params: 0
_________________________________________________________________


In [7]:
conv_base.trainable = False
print('This is the number of trainable weights before freezing the conv base:', len(model.trainable_weights))

model.compile(
    optimizer=optimizers.Adam(lr=5e-5), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=146, 
                              epochs=15, 
                              validation_data=validation_generator, 
                              validation_steps=31)

This is the number of trainable weights before freezing the conv base: 4
Epoch 1/15
146/146 [==============================] - 42s 286ms/step - loss: 1.7843 - categorical_accuracy: 0.3311 - val_loss: 1.4778 - val_categorical_accuracy: 0.4494
Epoch 2/15
146/146 [==============================] - 36s 250ms/step - loss: 1.2500 - categorical_accuracy: 0.5455 - val_loss: 1.4426 - val_categorical_accuracy: 0.4777
Epoch 3/15
146/146 [==============================] - 37s 251ms/step - loss: 1.0080 - categorical_accuracy: 0.6409 - val_loss: 1.3468 - val_categorical_accuracy: 0.5304
Epoch 4/15
146/146 [==============================] - 37s 254ms/step - loss: 0.8552 - categorical_accuracy: 0.7093 - val_loss: 1.1627 - val_categorical_accuracy: 0.5304
Epoch 5/15
146/146 [==============================] - 37s 256ms/step - loss: 0.7215 - categorical_accuracy: 0.7671 - val_loss: 1.1172 - val_categorical_accuracy: 0.5870
Epoch 6/15
146/146 [==============================] - 37s 255ms/step - loss: 0.638

In [8]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [9]:
model.compile(
    optimizer=optimizers.Adam(lr=1e-5), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=146, 
                              epochs=50, 
                              validation_data=validation_generator, 
                              validation_steps=31)

Epoch 1/50
146/146 [==============================] - 45s 305ms/step - loss: 0.1939 - categorical_accuracy: 0.9452 - val_loss: 0.6947 - val_categorical_accuracy: 0.7490
Epoch 2/50
146/146 [==============================] - 43s 292ms/step - loss: 0.0952 - categorical_accuracy: 0.9829 - val_loss: 0.6427 - val_categorical_accuracy: 0.7773
Epoch 3/50
146/146 [==============================] - 43s 293ms/step - loss: 0.0694 - categorical_accuracy: 0.9869 - val_loss: 0.7133 - val_categorical_accuracy: 0.7530
Epoch 4/50
146/146 [==============================] - 43s 293ms/step - loss: 0.0494 - categorical_accuracy: 0.9932 - val_loss: 0.6946 - val_categorical_accuracy: 0.7773
Epoch 5/50
146/146 [==============================] - 43s 294ms/step - loss: 0.0417 - categorical_accuracy: 0.9919 - val_loss: 0.6132 - val_categorical_accuracy: 0.8016
Epoch 6/50
146/146 [==============================] - 43s 296ms/step - loss: 0.0320 - categorical_accuracy: 0.9938 - val_loss: 0.6532 - val_categorical_acc

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc,  dashes=[2, 2, 10, 2], label='Training acc')
plt.plot(epochs, val_acc, 'g', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss,  dashes=[2, 2, 10, 2],label='Training losses')
plt.plot(epochs, val_loss, 'g', label='Validation losses')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model.save('fully_trained_model.h5')

# Train the full VGG network

In [ ]:
conv_base.trainable = True

model.compile(
    optimizer=optimizers.Adam(lr=1e-5), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=146, 
                              epochs=30, 
                              validation_data=validation_generator, 
                              validation_steps=31)

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc,  dashes=[2, 2, 10, 2], label='Training acc')
plt.plot(epochs, val_acc, 'g', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss,  dashes=[2, 2, 10, 2],label='Training losses')
plt.plot(epochs, val_loss, 'g', label='Validation losses')
plt.title('Training and validation loss')
plt.legend()

plt.show()